# Imports

In [30]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

In [31]:
from pandas_profiling import ProfileReport

In [32]:
conn = sqlite3.connect("data\data_wildfire_usa.sqlite")

In [33]:
fires = pd.read_sql_query("SELECT * FROM fires", conn)
# fires est un tableau comprenant des données sur les incendies de forêt pour la période 1992-2015
# compilées à partir des systèmes de déclaration fédéraux, étatiques et locaux des États-Unis.

In [34]:
fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1880465 entries, 0 to 1880464
Data columns (total 39 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   OBJECTID                    int64  
 1   FOD_ID                      int64  
 2   FPA_ID                      object 
 3   SOURCE_SYSTEM_TYPE          object 
 4   SOURCE_SYSTEM               object 
 5   NWCG_REPORTING_AGENCY       object 
 6   NWCG_REPORTING_UNIT_ID      object 
 7   NWCG_REPORTING_UNIT_NAME    object 
 8   SOURCE_REPORTING_UNIT       object 
 9   SOURCE_REPORTING_UNIT_NAME  object 
 10  LOCAL_FIRE_REPORT_ID        object 
 11  LOCAL_INCIDENT_ID           object 
 12  FIRE_CODE                   object 
 13  FIRE_NAME                   object 
 14  ICS_209_INCIDENT_NUMBER     object 
 15  ICS_209_NAME                object 
 16  MTBS_ID                     object 
 17  MTBS_FIRE_NAME              object 
 18  COMPLEX_NAME                object 
 19  FIRE_YEAR            

In [35]:
fires.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...


# Description des variables + notes à chaud

1. OBJECTID : ID **=> Donnée non nécessaire pour notre étude**

2.   FOD_ID : Identificateur unique global qui ressemble à l'OBJECTID **=> A FAIRE : Vérifier si OBJECTID = FOD_ID pour en supprimer un des deux**

3.   FPA_ID : Identificateur unique nécessaires pour remonter jusqu'à l'enregistrement d'origine 

4.   SOURCE_SYSTEM_TYPE : Type de données source ou système d'où provient l'enregistrement (fédéral, non fédéral ou interagence) 

5.   SOURCE_SYSTEM : Nom ou autre identifiant de la données source ou système d'où provient l'enregistrement 

**A FAIRE : Comparer les 2 variables SOURCE_SYSTEM_TYPE et SOURCE_SYSTEM pour voir si on ne peut conserver qu'une des deux.**

6.   NWCG_REPORTING_AGENCY : Active National Wildlife Coordinating Group = Groupe national de coordination de la faune. Identifiant d'unité pour l'agence qui prépare le rapport d'incendie.
  *   BIA = Bureau of Indian Affairs 
  *   BLM = Bureau of Land Management
  *   BOR = Bureau of Reclamation
  *   DOD = Department of Defense
  *   DOE = Department of Energy
  *   FS = Forest Service
  *   FWS = Fish and Wildlife Service
  *   IA = Interagency Organization
  *   NPS = National Park Service
  *   ST/C&L = State, County, or Local Organization
  *   TRIBE = Tribal Organization

7.   NWCG_REPORTING_UNIT_ID : Identifiant d'unité NWCG actif pour l'unité préparant le rapport d'incendie **=> Donnée non nécessaire si on a le nom**

8.   NWCG_REPORTING_UNIT_NAME : Nom d'unité NWCG actif pour l'unité préparant le rapport d'incendie

**=> A FAIRE : Comparer NWCG_REPORTING_UNIT_ID et NWCG_REPORTING_UNIT_NAME pour voir si les données se recoupent entre elles et ne garder qu'une des deux variables**

9.   SOURCE_REPORTING_UNIT : Code de l'unité d'agence préparant le rapport d'incendie, basé sur le code/nom dans les données source **=> Donnée non nécessaire si on a le nom**

10.  SOURCE_REPORTING_UNIT_NAME : Nom de l'unité de l'agence déclarante préparant le rapport d'incendie, basé sur le code/nom des données source **A FAIRE : vérifier si NWCG_REPORTING_UNIT_NAME = SOURCE_REPORTING_UNIT_NAME our n'en garder qu'un des deux**

11.  LOCAL_FIRE_REPORT_ID : Numéro ou code qui identifie de manière unique un rapport d'incident pour une unité de déclaration particulière et une année civile particulière **A FAIRE : vérifier s'il y a des doublons**

12.  LOCAL_INCIDENT_ID : Numéro ou code qui identifie de manière unique un incident pour une organisation locale de gestion des incendies particulière au cours d'une année civile donnée **A FAIRE : vérifier s'il y a des doublons**

13.  FIRE_CODE : code utilisé au sein de la communauté inter-agences des feux de forêt pour suivre et compiler les informations sur les coûts de la suppression des incendies d'urgence **=> Donnée non nécessaire si on a le nom. Nécessaire seulement si cette donnée nous permet de combler des valeurs manquantes dans la colonne FIRE_NAME**

14.  FIRE_NAME : Nom de l'incident, tiré du rapport d'incendie (primaire) ou du rapport ICS-209 (secondaire) **A FAIRE : comparer les variables FIRE_CODE et FIRE_NAME.**

15.  ICS_209_INCIDENT_NUMBER : Identifiant d'incident (événement), issu du rapport ICS-209. **A FAIRE : à voir si nécessaire**

16.  ICS_209_NAME : Nom de l'incident, tiré du rapport ICS-209. **A FAIRE : à voir si nécessaire**

17.  MTBS_ID : Identifiant d'incident, à partir de l'ensemble de données de périmètre MTBS. **=> Donnée non nécessaire si on a le nom**

18.  MTBS_FIRE_NAME : Nom de l'incident, issu de l'ensemble de données de périmètre MTBS. **A FAIRE : à voir si nécessaire**

19.  COMPLEX_NAME : Nom du complexe sous lequel l'incendie a finalement été maîtrisé, lorsqu'il est discernable. **A FAIRE : à voir si nécessaire**

20.  FIRE_YEAR : Année civile au cours de laquelle l'incendie a été découvert ou confirmé. **A FAIRE : vérifier le format de date (à l'air dêtre conforme au format yyyy)**

21.  DISCOVERY_DATE : Date à laquelle l'incendie a été découvert ou confirmé. **A FAIRE : vérifier le format de date : le format a l'air particulier. A creuser.**

22.  DISCOVERY_DOY : Jour de l'année où l'incendie a été découvert ou confirmé. **A FAIRE : vérifier le format de date + renommer DISCOVERY_DOY en DISCOVERY_DAY**

23.  DISCOVERY_TIME : Heure de la journée à laquelle l'incendie a été découvert ou confirmé. **A FAIRE : vérifier le format + creuser l'analyse par rapport aux horaires de feux**
            
24.  STAT_CAUSE_CODE : Code de la cause (statistique) de l'incendie. **=> Donnée non nécessaire si on a le nom. Nécessaire seulement si cette donnée nous permet de combler des valeurs manquantes dans la colonne STAT_CAUSE_DESCR**

25.  STAT_CAUSE_DESCR : Description de la cause (statistique) de l'incendie. 

26.  CONT_DATE : Date à laquelle l'incendie a été déclaré maîtrisé ou autrement maîtrisé (mm/jj/aaaa où mm=mois, jj=jour et aaaa=année). **A FAIRE : vérifier le format de date : le format a l'air particulier. A creuser.**

27.  CONT_DOY : Jour de l'année où l'incendie a été déclaré maîtrisé ou autrement maîtrisé. **A FAIRE : vérifier le format de date + renommer CONT_DOY en CONT_DAY**

28.  CONT_TIME : Heure de la journée à laquelle l'incendie a été déclaré maîtrisé ou maîtrisé (hhmm où hh=heure, mm=minutes). **Le format a l'air conforme à l'explication hhmm. A FAIRE : voir si la variable DISCOVERY_TIME a le même format.**

29.  FIRE_SIZE : Estimation des acres dans le périmètre final du feu. (1 acre anglo-saxon = 40,47 ares). **A FAIRE : voir s'il y a corrélation entre la taille du feu et le temps**

30.  FIRE_SIZE_CLASS : Code de la taille du feu basé sur le nombre d'acres dans les dépenses finales du périmètre d'incendie
  *   A = > 0 et <= 0,25 acre
  *   B = 0,26 à 9,9 acres
  *   C = 10,0 à 99,9 acres
  *   D = 100 à 299 acres
  *   E = 300 à 999 acres
  *   F = 1000 à 4999 acres
  *   G = 5000+ acres

31.  LATITUDE : Latitude (NAD83) pour l'emplacement ponctuel de l'incendie (degrés décimaux). **A FAIRE : vérifier si les coordonnées sont réelles et si elles ont toutes le même format.**

32.  LONGITUDE : Longitude (NAD83) pour l'emplacement ponctuel de l'incendie (degrés décimaux). **A FAIRE : vérifier si les coordonnées sont réelles et si elles ont toutes le même format.**

33.  OWNER_CODE : Code du propriétaire principal ou de l'entité responsable de la gestion du terrain au point d'origine de l'incendie au moment de l'incident. **=> Donnée non nécessaire si on a le nom. Nécessaire seulement si cette donnée nous permet de combler des valeurs manquantes dans la colonne OWNER_DESCR**

34.  OWNER_DESCR : Nom du propriétaire principal ou de l'entité responsable de la gestion du terrain au point d'origine de l'incendie au moment de l'incident

35.  STATE : Code alphabétique à deux lettres de l'état dans lequel l'incendie a brûlé (ou s'est déclaré), basé sur la désignation nominale dans le rapport d'incendie. **Correspondance à faire avec la liste ci-dessous**

(AK=Alaska, AL=Alabama, AR=Arkansas, AZ=Arizona, CA=California, CO=Colorado, CT=Connecticut, DE=Delaware, DHS=Department of Homeland Security, DOC= Department of Commerce, DOD=Department of Defense, DOE=Department of Energy, DOI= Department of Interior, DOL=Department of Labor, FL=Florida, GA=Georgia, IA=Iowa, IA/GC=Non-Departmental Agencies, ID=Idaho, IL=Illinois, IN=Indiana, KS=Kansas, KY=Kentucky, LA=Louisiana, MA=Massachusetts, MD=Maryland, ME=Maine, MI=Michigan, MN=Minnesota, MO=Missouri, MS=Mississippi, MT=Montana, NC=North Carolina, NE=Nebraska, NG=Non-Government, NH=New Hampshire, NJ=New Jersey, NM=New Mexico, NV=Nevada, NY=New York, OH=Ohio, OK=Oklahoma, OR=Oregon, PA=Pennsylvania, PR=Puerto Rico, RI=Rhode Island, SC=South Carolina, SD=South Dakota, ST/L=State or Local Government, TN=Tennessee, Tribe=Tribe, TX=Texas, USDA=Department of Agriculture, UT=Utah, VA=Virginia, VI=U. S. Virgin Islands, VT=Vermont, WA=Washington, WI=Wisconsin, WV=West Virginia, WY=Wyoming).

36.  COUNTY : Comté, ou équivalent, dans lequel l'incendie s'est déclaré (ou a pris naissance), en fonction de la désignation nominale dans le rapport d'incendie.

37.  FIPS_CODE : Code à trois chiffres de la publication FIPS (Federal Information Process Standards) 6-4 pour la représentation des comtés et des entités équivalentes. **=> Donnée non nécessaire si on a le nom. Nécessaire seulement si cette donnée nous permet de combler des valeurs manquantes dans la colonne FIPS_NAME**

38.  FIPS_NAME : Nom du comté de la publication FIPS 6-4 pour la représentation des comtés et des entités équivalentes

39.  Shape : donnée inutile

# Statistiques

In [36]:
# Statistiques au format html
#prof = ProfileReport(fires)
#prof.to_file(output_file='rapport.html')

In [37]:
# Affichage des statistiques dans le notebook via la librairie pandas_profiling
#profile = ProfileReport(fires, title='Analyse du fichier WildFire', html={'style':{'full_width':True}})
#profile.to_notebook_iframe()

# Data Cleaning

In [114]:
# Le chargement du fichier initial est nécessaire dans cette cellule pour éviter tout message d'erreur si la cellule est ré-exécutée
fires = pd.read_sql_query("SELECT * FROM fires", conn)
# Suppression de la colonne Shape
fires = fires.drop(['Shape'], axis = 1)
# Renommage des colonnes : DAY à la place de DOY
fires = fires.rename(columns = {'DISCOVERY_DOY' : 'DISCOVERY_DAY', 'CONT_DOY' : 'CONT_DAY'})
fires.head()


,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,0.10,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,0.25,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,0.10,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,0.10,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,0.10,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine


Suite à la méthode ProfileReport, nous pouvons déclarer que :
- 27% des cellules du DataFrame sont manquantes
- étant donné la part de valeurs manquantes dans les colonnes ICS_209_INCIDENT_NUMBER (98.6%), ICS_209_NAME (98.6%), MTBS_ID (99.4%), MTBS_FIRE_NAME (99.4%), COMPLEX_NAME (99.7%), ces colonnes ne sont pas nécessaires
- les colonnes OBJECTID et FOD_ID ont des valeurs uniques
- 

## Etude des variables identiques ou similaires

In [38]:
# Est ce que la variable OBJECTID = FOD_ID ?
(fires.OBJECTID == fires.FOD_ID).value_counts()
# Les 2 colonnes semblent être identiques mais l'ID de FOD_ID est décalé à partir de l'index 72. 
# A priori ces données ne sont pas nécessaires pour l'étude. On ne s'en préoccupera pas.

False    1880393
True          72
dtype: int64

In [135]:
# Est ce qu'une valeur donnée dans la colonne SOURCE_SYSTEM_TYPE correspond à une valeur spécifique de la colonne SOURCE_SYSTEM ?
source_system = (fires['SOURCE_SYSTEM_TYPE']) + '_' + (fires['SOURCE_SYSTEM'])
print(source_system.duplicated().value_counts(), '\n')
print(source_system.drop_duplicates(keep = 'first').values)
# On voit bien que 39 observations ne sont pas identiques. On peut se demander s'il s'agit d'erreurs.

True     1880426
False         39
dtype: int64 

['FED_FS-FIRESTAT' 'FED_DOI-WFMI' 'FED_FWS-FMIS' 'NONFED_FA-NFIRS'
 'NONFED_ST-NASF' 'NONFED_ST-AZAZS' 'NONFED_ST-MOMOS' 'INTERAGCY_IA-AKACC'
 'NONFED_ST-MTMTS' 'NONFED_ST-SCSCS' 'NONFED_ST-COCOS' 'NONFED_ST-MEMES'
 'NONFED_ST-ORORS' 'NONFED_ST-MSMSS' 'NONFED_ST-NMNMS' 'NONFED_ST-SDSDS'
 'NONFED_ST-UTUTS' 'NONFED_ST-WAWAS' 'NONFED_ST-WYWYS' 'NONFED_ST-OKOKS'
 'NONFED_ST-WIWIS' 'NONFED_ST-MIMIS' 'NONFED_ST-KYKYS' 'NONFED_ST-ALALS'
 'NONFED_ST-ARARS' 'NONFED_ST-GAGAS' 'NONFED_ST-LALAS' 'NONFED_ST-TNTNS'
 'NONFED_ST-VAVAS' 'NONFED_ST-FLFLS' 'NONFED_ST-NCNCS'
 'INTERAGCY_IA-ICS209' 'NONFED_ST-CACDF' 'NONFED_ST-CTCTS'
 'NONFED_ST-NENES' 'NONFED_IA-ICS209' 'NONFED_ST-TXTXS'
 'INTERAGCY_IA-PRIITF' 'INTERAGCY_IA-HIWMO']


In [180]:
# Est ce qu'une valeur donnée dans la colonne NWCG_REPORTING_UNIT_ID correspond à une valeur spécifique de la colonne NWCG_REPORTING_UNIT_NAME ?
nwcg_reporting = pd.DataFrame(fires['NWCG_REPORTING_UNIT_ID']).join(fires['NWCG_REPORTING_UNIT_NAME'])
nwcg_reporting.duplicated().value_counts()
# On voit bien que 1640 observations ne sont pas identiques. On peut se demander s'il s'agit d'erreur.

True     1878825
False       1640
dtype: int64

## Examen des doublons

L'examen des doublons sur l'ensemble des colonnes du DataFrame peut se faire directement avec les résulats de ProfileReport. Cependant, pour plus de clareté nous allons détailler certains points

In [39]:
# Création d'un dictionnaire regroupant les colonnes et le nombre de doublons par colonne
doublons = {}
for col in fires.columns :
    duplicates = fires[col].duplicated().sum()
    doublons[col] = duplicates

doublons

{'OBJECTID': 0,
 'FOD_ID': 0,
 'FPA_ID': 3,
 'SOURCE_SYSTEM_TYPE': 1880462,
 'SOURCE_SYSTEM': 1880427,
 'NWCG_REPORTING_AGENCY': 1880454,
 'NWCG_REPORTING_UNIT_ID': 1878825,
 'NWCG_REPORTING_UNIT_NAME': 1878830,
 'SOURCE_REPORTING_UNIT': 1875473,
 'SOURCE_REPORTING_UNIT_NAME': 1876024,
 'LOCAL_FIRE_REPORT_ID': 1866956,
 'LOCAL_INCIDENT_ID': 1314550,
 'FIRE_CODE': 1708018,
 'FIRE_NAME': 1386831,
 'ICS_209_INCIDENT_NUMBER': 1857727,
 'ICS_209_NAME': 1860891,
 'MTBS_ID': 1869983,
 'MTBS_FIRE_NAME': 1872331,
 'COMPLEX_NAME': 1879048,
 'FIRE_YEAR': 1880441,
 'DISCOVERY_DATE': 1871699,
 'DISCOVERY_DOY': 1880099,
 'DISCOVERY_TIME': 1879024,
 'STAT_CAUSE_CODE': 1880452,
 'STAT_CAUSE_DESCR': 1880452,
 'CONT_DATE': 1871704,
 'CONT_DOY': 1880098,
 'CONT_TIME': 1879023,
 'FIRE_SIZE': 1866828,
 'FIRE_SIZE_CLASS': 1880458,
 'LATITUDE': 986404,
 'LONGITUDE': 882929,
 'OWNER_CODE': 1880449,
 'OWNER_DESCR': 1880449,
 'STATE': 1880413,
 'COUNTY': 1877009,
 'FIPS_CODE': 1880179,
 'FIPS_NAME': 1878766,
 '

Le résultat de ce dictionnaire nous indique plusieurs choses :
1. Pour la plupart des variables, il n'existe que très peu de données distinctes car il y a beaucoup de doublons.
2. Pour les variables uniques :
    - telles que OBJECTID et FOD_ID, il n'existe pas de doublons. Il n'y a donc pas d'erreur dans ces données car elles sont uniques (en lien avec les résultats de la méthode ProfileReport).
    - En revanche, pour la variable FPA_ID, nous remarquons qu'il y a 3 doublons. Ces erreurs doivent être corrigées car FPA_ID est une donnée censée être unique (selon le ProfileReport, FPA_ID possède 99.9% de valeurs distinctes).

In [181]:
# Identification et correction des doublons de FPA_ID
fires[fires['FPA_ID'].duplicated() == True]


,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME
1565829,1565830,201432072,FS-1452833,FED,FS-FIRESTAT,FS,USNMCIF,Cibola National Forest,0303,Cibola National Forest,...,0.10,A,35.337222,-107.779444,5.0,USFS,NM,031,031,McKinley
1634979,1634980,201750002,ICS209_2009_KS-DDQ-128,INTERAGCY,IA-ICS209,DOD,USKSFRQ,Fort Riley Kansas,KSDDQ,Fort Riley,...,2200.00,F,39.220000,-96.940000,6.0,OTHER FEDERAL,KS,Riley,161,Riley
1870332,1870333,300306586,SFO-2015CACDFLNU003791,NONFED,ST-NASF,ST/C&L,USCALNU,Sonoma-Lake Napa Unit,CALNU,Sonoma-Lake Napa Unit,...,0.01,A,38.342004,-121.958596,14.0,MISSING/NOT SPECIFIED,CA,SOLANO,095,Solano


In [59]:
# A FAIRE !!!!!!!!!!!!!!!!!
FS_1452833 = fires[fires['FPA_ID'] == 'FS-1452833']
FS_1452833[]

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
21986,21987,22093,FS-1452833,FED,FS-FIRESTAT,FS,USNMCIF,Cibola National Forest,0303,Cibola National Forest,...,B,35.312778,-107.593056,5.0,USFS,NM,31,031,McKinley,b'\x00\x01\xad\x10\x00\x00L\xbaN\x9f\xf4\xe5Z\...
1565829,1565830,201432072,FS-1452833,FED,FS-FIRESTAT,FS,USNMCIF,Cibola National Forest,0303,Cibola National Forest,...,A,35.337222,-107.779444,5.0,USFS,NM,031,031,McKinley,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\xf1Z\...


### Examen des variables ayant au moins 1 800 000 doublons

Le chiffre de 1 800 000 a été retenu car il existe 1 880 464 entrées dans le dataframe fires. Il peut être intéressant de regarder de plus près quelles sont les données présentes dans ces variables.

In [42]:
# condition sur le dictionnaire doublons

In [43]:
# Pour chaque colonne du dataframe nous regardons combien il y a de valeurs uniques.
#for col in fires.columns :
    #fires[col].value_counts()

## Examen du type/format de données

Le champ DISCOVERY_DATE est au format de date julienne. Je veux convertir ce champ en dates grégoriennes. 
Je peux ensuite utiliser la date grégorienne pour obtenir des champs de données supplémentaires - mois et jour de la semaine.

In [44]:
# Changement du format de date
#fires['DATE'] = pd.to_datetime(fires['DISCOVERY_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
# Vérification du format
#print(fires['DISCOVERY_DATE'], fires['DATE'])

In [45]:
#fires['MONTH'] = pd.DatetimeIndex(fires['DATE']).month
#fires['DAY_OF_WEEK'] = fires['DATE'].dt.day_name()
#print(fires.head())

# Examen des causes des incendies

In [46]:
#plt.figure(figsize=(10, 8))
#plt.barh(fires['STAT_CAUSE_DESCR'].value_counts(), color = 'darkslateblue', width = 0.6)
#plt.title(label = 'XXXX', size = 'xx-large', weight = 'bold', color = 'crimson')
#plt.xlabel('XXXX', weight = 'bold')
#plt.show()

Il y a 13 causes répertoriées dans l'ensemble de données. 
Les incendies sont-ils plus probables certains jours de la semaine ?

In [47]:
#fires['DAY_OF_WEEK'].value_counts().plot(kind='barh',color='navy')
#plt.show()

Les incendies sont légèrement plus susceptibles de se déclencher le week-end. 
On peut supposer que les incendies sont déclenchés par des personnes négligentes : feux de camp, cigarettes, incendies malveillants.
Il est peu probable que les incendies augmentant le week-end proviennent de causes naturelles, comme la foudre par exemple. Nous pouvons tester cela :

In [48]:
#fires_lightning = fires[fires['STAT_CAUSE_DESCR']=='Lightning']
#fires_lightning['DAY_OF_WEEK'].value_counts().plot(kind='barh',color='navy')
#plt.show()

Il y a quelques petites variations, mais nous voyons que les feux se déclenchant le week-end sont moins élevés. 
Nous pouvons examiner si l'augmentation générale du week-end est due à des incendies malveillants
Incendie criminel :

In [49]:
#fires_arson = fires[fires['STAT_CAUSE_DESCR']=='Arson']
#fires_arson['DAY_OF_WEEK'].value_counts().plot(kind='barh',color='navy')
#plt.show()

Cela montre une tendance plus significative.
Les incendies criminels sont plus probables le week-end que pendant la semaine, cela représente une augmentation d'environ 30 % de la moyenne des jours de la semaine. 

Quels États ont le plus d'incendies ? (pour éviter un trop grand nombre de données, nous pouvons nous concentrer sur les dix premiers)

In [50]:
#fires['STATE'].value_counts().head(n=10).plot(kind='barh',color='navy')
#plt.show()

CA et TX sont dans le top 3, ce sont de grands états avec des climats secs. En revanche, GA est également dans le top 3 on peut se demander pourquoi
Nous pouvons enquêter sur les causes des incendies dans ces 3 états et rechercher des différences. On extrait les données pour ces 3 états :

In [51]:
#df_CA = df[df['STATE']=='CA']
#df_GA = df[df['STATE']=='GA']
#df_TX = df[df['STATE']=='TX']

In [52]:
#fires_CA['STAT_CAUSE_DESCR'].value_counts().plot(kind='barh',color='navy',title='causes of fires for CA')
#plt.show()

In [53]:
#fires_GA['STAT_CAUSE_DESCR'].value_counts().plot(kind='barh',color='navy',title='causes of fires for GA')
#plt.show()

In [54]:
#fires_TX['STAT_CAUSE_DESCR'].value_counts().plot(kind='barh',color='navy',title='causes of fires for TX')
#plt.show()

Il existe des différences entre les états. 
Il existe deux manières de déterminer l'emplacement d'un incendie dans ce jeu de données - ETAT ou LATITUDE + LONGITUDE
Nous pouvons avoir une idée de la couverture géographique des données en traçant chaque paire de latitude et de longitude

In [55]:
#fires.plot(kind='scatter',x='LONGITUDE',y='LATITUDE',color='coral',alpha=0.3)
#plt.show()